In [2]:
! python --version

Python 3.8.10


In [1]:
####################################
#
#  ADD THIS TO EVERY COLAB FILE!
#
####################################

#!pip install -q import-ipynb

from google.colab import drive
drive.mount('/content/drive')

#import import_ipynb
import drive.Shareddrives.GPTJ.project.settings as settings

PATH_PROJECT = settings.PATH_PROJECT
PATH_DATA = settings.PATH_DATA

! cd $PATH_PROJECT && pip install -q -r requirements.txt

Mounted at /content/drive
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.0/144.0 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.7 MB/s eta 0

In [ ]:
!nvidia-smi

Sat Jan 21 23:17:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, GPTNeoForCausalLM

In [ ]:
# MODEL

#model_name = 'EleutherAI/gpt-neo-1.3B' # CUDA out of memory.

model_name = 'gpt2' # GPU vRAM 3495MiB - 3.5G
#model_name = 'gpt2-medium' # GPU vRAM 7389MiB - 7.3G
#model_name = 'gpt2-large' # GPU vRAM 14797MiB - 14.8G

#model_name = 'gpt2-xl' # CUDA out of memory.


In [ ]:
# Set the random seed to a fixed value to get reproducible results 
torch.manual_seed(42)

tokenizer = GPT2Tokenizer.from_pretrained(model_name, 
    bos_token='<|startoftext|>',
    eos_token='<|endoftext|>', 
    pad_token='<|pad|>'
)
model = GPT2LMHeadModel.from_pretrained(model_name).cuda()
model.resize_token_embeddings(len(tokenizer))

torch.save(model, os.path.join(PATH_DATA, model_name, 'model'))

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
descriptions = pd.read_csv(PATH_PROJECT + '/netflix_titles.csv')['description']
max_length = max([len(tokenizer.encode(description)) for description in descriptions])

max_length, descriptions

(62, 0                            As her father nears the end.
 1       As her father nears the end of his life, filmm...
 2       After crossing paths at a party, a Cape Town t...
 3       To protect his family from a powerful drug lor...
 4       Feuds, flirtations and toilet talk go down amo...
                               ...                        
 8803    A political cartoonist, a crime reporter and a...
 8804    While living alone in a spooky town, a young g...
 8805    Looking to survive in a world taken over by zo...
 8806    Dragged from civilian life, a former superhero...
 8807    A scrappy but poor boy worms his way into a ty...
 Name: description, Length: 8808, dtype: object)

In [ ]:
class NetflixDataset(Dataset):

    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>' 
                                        + txt +    
                                        '<|endoftext|>',
                                        truncation=True,
                                        max_length=max_length, 
                                        padding='max_length')
            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

dataset = NetflixDataset(descriptions, tokenizer, max_length)

print(len(dataset))

train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

8808


In [ ]:
# Here I will pass the output directory where 
# the model predictions and checkpoints will be stored, 
# batch sizes for the training and validation steps, 
# and warmup_steps to gradually increase the learning rate
training_args = TrainingArguments(
    output_dir=os.path.join(PATH_DATA, model_name, 'partial'),
    num_train_epochs=5,
    logging_steps=5000,
    save_steps=5000,                                   
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=100,
    weight_decay=0.01,  
    logging_dir=os.path.join(PATH_DATA, model_name, 'logs')
)

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    # This custom collate function is necessary 
    # to built batches of data
    data_collator=lambda data: {
        'input_ids': torch.stack([f[0] for f in data]),       
        'attention_mask': torch.stack([f[1] for f in data]),
        'labels': torch.stack([f[0] for f in data])
    }
)
# Start training process!
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7927
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 19820
  Number of trainable parameters = 124441344


Step,Training Loss
5000,2.117800
10000,1.534200
15000,1.344600


Saving model checkpoint to /content/drive/Shareddrives/GPTJ/data/gpt2/partial/checkpoint-5000
Configuration saved in /content/drive/Shareddrives/GPTJ/data/gpt2/partial/checkpoint-5000/config.json
Model weights saved in /content/drive/Shareddrives/GPTJ/data/gpt2/partial/checkpoint-5000/pytorch_model.bin
Saving model checkpoint to /content/drive/Shareddrives/GPTJ/data/gpt2/partial/checkpoint-10000
Configuration saved in /content/drive/Shareddrives/GPTJ/data/gpt2/partial/checkpoint-10000/config.json
Model weights saved in /content/drive/Shareddrives/GPTJ/data/gpt2/partial/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to /content/drive/Shareddrives/GPTJ/data/gpt2/partial/checkpoint-15000
Configuration saved in /content/drive/Shareddrives/GPTJ/data/gpt2/partial/checkpoint-15000/config.json
Model weights saved in /content/drive/Shareddrives/GPTJ/data/gpt2/partial/checkpoint-15000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =

TrainOutput(global_step=19820, training_loss=1.5545982691883198, metrics={'train_runtime': 1984.4301, 'train_samples_per_second': 19.973, 'train_steps_per_second': 9.988, 'total_flos': 1254084376320000.0, 'train_loss': 1.5545982691883198, 'epoch': 5.0})

In [ ]:
trainer.save_model(os.path.join(PATH_DATA, model_name, 'model-trained'))

Saving model checkpoint to /content/drive/Shareddrives/GPTJ/data/gpt2/model-trained
Configuration saved in /content/drive/Shareddrives/GPTJ/data/gpt2/model-trained/config.json
Model weights saved in /content/drive/Shareddrives/GPTJ/data/gpt2/model-trained/pytorch_model.bin


In [ ]:

# Start every description with a special BOS token
generated = tokenizer('<|startoftext|>', return_tensors='pt').input_ids.cuda()

# Start every description with a special BOS token
#prompt = 'mov ebp var_A dx' # mov edx ebp var_E'
#prompt = 'An apple is'
#generated = tokenizer(prompt, return_tensors='pt').input_ids.cuda()

# Generate 3 movie descriptions
sample_outputs = model.generate(generated, 
    # Use sampling instead of greedy decoding 
    do_sample=True, 
    # Keep only top 50 token with the highest probability
    top_k=50, 
    # Maximum sequence length
    max_length=50,
    #max_new_tokens=100,
    # Keep only the most probable tokens with cumulative probability of 95%
    top_p=0.95, 
    # Changes randomness of generated sequences to 1.9
    temperature=0.01,
    # Number of sequences to generate                 
    num_return_sequences=10
)

# Print generated descriptions
for i, sample_output in enumerate(sample_outputs): 
    print('{}: {}'.format(i, tokenizer.decode(sample_output, skip_special_tokens=True)).replace('\n', ' '))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.0.dev0"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: A young man's life is turned upside down when he is forced to work as a house servant for a corrupt local government official.
1: A young man's attempt to steal a fortune from a wealthy family is thwarted by a corrupt politician, who has other plans.
2: A young man and his family are forced to move into a house that has been turned into a gambling densitium.
3: A young man's life is turned upside down when he meets a woman who offers to help him with his problems and help him find a way to live.
4: A young woman's life is turned upside down when she falls for a man who's trying to make her fall for his greedy boss.
5: A young man's love for his girlfriend is tested when he meets a woman who is willing to do anything to get him married.
6: A young man is forced to live with his father in a small town until his father's death, which brings new possibilities for his life and love.
7: A young man with a knack for drawing finds himself in a love triangle with a young woman who's about to

In [ ]:
!nvidia-smi

Sun Jan 22 00:02:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    35W /  70W |   3660MiB / 15109MiB |     25%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------